Use cellchat to classify samples

In [ ]:
suppressPackageStartupMessages({
    library(CellChat, quietly = T)
    library(patchwork, quietly = T)
    library(RhpcBLASctl, quietly = T)
    library(Matrix, quietly = T)
    library(rhdf5, quietly = T)
    library(stringr, quietly = T)
})
options(stringsAsFactors = FALSE)
# expression_data_path = '/data2/hratch/immune_CCI/covid/covid_atlas/interim/umi_for_timing/'
input_path = '/data2/hratch/immune_CCI/covid/covid_atlas/interim/classification_inputs/'

RhpcBLASctl::blas_set_num_threads(20) 

In [ ]:
# parameters
cell_grouper<-'majorType'
seed = 888
# inputs
group = FALSE
sample.names<-c('S-S083', 'S-M001', 'S-HC018-1')

Load data:

In [ ]:
# load metadata
md.cell <- read.csv(paste0(input_path,'metadata.csv'), row.names = 1)

sample_names<-unique(md.cell$sampleID)
sample_context_map<-list()
for (sn in sample_names){
    context<-unique(md.cell[md.cell$sampleID == sn,'CoVID.19.severity'])
    if (length(context) != 1){stop('Incorred sample to context mapping')}
    sample_context_map[[sn]]<-context
}
contexts<-unique(sample_context_map)
# load LR pairs
# filter for the LR pairs used by Tensor cell2cell
# lr_pairs<-read.csv(paste0(input_data_path,'Tensor-cell2cell-LRpairs.csv'))
# lr_pairs<-lr_pairs$interaction_name
humandb<-CellChatDB.human
# humandb$interaction<-CellChatDB.human$interaction[CellChatDB.human$interaction$interaction_name %in% lr_pairs, ] 
# saveRDS(humandb, paste0(output_results_path, 'humandb.rds'))

In [ ]:
# load the UMI counts
read_sample_h5<-function(sn){
    counts<-h5read(paste0(input_path, 'umi_per_sample.h5'), sn)
    count<-counts[[4]]
    colnames(count)<-counts[[2]]
    rownames(count)<-sapply(counts[[1]], function(x) str_replace_all(x, '-', '.')) 
    return(count)
}


if (!group){
    counts<-lapply(setNames(sample.names, sample.names), function(sn) read_sample_h5(sn))
}else{ # group by context
    by.context<-lapply(setNames(contexts, contexts), function(context) names(sample_context_map[sample.names][sample_context_map[sample.names] == context]))
    
    group_by_context<-function(context){
        sns<-by.context[[context]]       
        counts<-lapply(sns, function(sn) read_sample_h5(sn))    
        counts<-do.call(cbind, counts)
        return (counts)
    }
    counts<-lapply(setNames(contexts, contexts), function(context) group_by_context(context))                   
                    
}

Run cellchat:

In [ ]:
# create cellchat object for each sample or sample.name
covid.list<-list()
for (sample.name in names(counts)){
    # loop through each sample.name and create a cell type future
    expr<-CellChat::normalizeData(counts[[sample.name]])
    cellchat<-createCellChat(object = as(expr, "dgCMatrix"), meta = md.cell[colnames(expr),], 
                                   group.by = cell_grouper)
    cellchat@DB <- humandb # human organism

    cellchat <- subsetData(cellchat) # subset the expression data of signaling genes, assign to @data.signalling 
    cellchat <- identifyOverExpressedGenes(cellchat)
    cellchat <- identifyOverExpressedInteractions(cellchat) # generate @ LR slot used by computeCommunProb
    cellchat <- projectData(cellchat, PPI.human) # shallow sequencing depth
    
    cellchat <- computeCommunProb(cellchat, raw.use = F, type = 'triMean', trim = NULL, seed.use = seed, 
                                 population.size = F) 
    
    # The functional similarity analysis requires the same cell population composition between two datasets.
    cellchat <- filterCommunication(cellchat, min.cells = 10)
    cellchat <- computeCommunProbPathway(cellchat)
    covid.list[[sample.name]]<-cellchat
}

# merge and analyze
cellchat <- mergeCellChat(covid.list, add.names = names(covid.list))
cellchat <- computeNetSimilarityPairwise(cellchat, type = 'structural')
cellchat <- netEmbedding(cellchat, type = 'structural')   